In [11]:
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import dask.array as da
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
from scipy.spatial.distance import cdist
from pathlib import Path

In [288]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55862 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55862/status,
Dashboard: http://127.0.0.1:55862/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55865,Workers: 1
Dashboard: http://127.0.0.1:55862/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:55872,Total threads: 4
Dashboard: http://127.0.0.1:55873/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:55868,


In [238]:
output_dask_dir = Path('flattened_images/')
output_dask_dir.mkdir(parents=True, exist_ok=True)
_ddf = dd.read_csv(output_dask_dir/'ddf*.csv')

In [239]:
_ddf.head()

,flattened_image,ID
0,[255 255 255 ... 255 255 255],10000
1,[255 255 255 ... 255 255 255],10001
2,[255 255 255 ... 255 255 255],10002
3,[255 255 255 ... 255 255 255],10003
4,[255 255 255 ... 255 255 255],10004


In [240]:
num_partition = _ddf.npartitions
print(num_partition)

10


In [318]:
def find_most_similar(chunk):
    vectors = np.array(chunk['flattened_image'].tolist())
    distances = cdist(vectors, vectors, metric='euclidean')
    np.fill_diagonal(distances, np.inf)
    closest_indices = np.argmin(distances, axis=1)
    closest_ids = chunk.iloc[closest_indices]['ID'].tolist()
    closest_distances = distances[np.arange(len(distances)), closest_indices]
    
    result = pd.DataFrame({'closest_ID': closest_ids, 'distance': closest_distances}, index=chunk.index)

    return result

In [271]:

def mapper(chunk, partition_info):
    chunk_number = partition_info['number']
    
    new_rows = []
    for i in range(num_partition):
        if i != chunk_number:
            key = tuple(sorted((i, chunk_number)))
            for index, row in chunk.iterrows():
                new_row = {'key': key, 'flattened_image': row['flattened_image'], 'ID': row['ID']}
                new_rows.append(new_row)

    return pd.DataFrame(new_rows, columns=['key', 'flattened_image', 'ID'])

In [322]:
def reducer(chunk):
    image_ids = chunk['ID']
    closest_ids_distances = find_most_similar(chunk)
    
    # data = list(zip(image_ids, closest_ids_distances))
    data = pd.concat([image_ids, closest_ids_distances], axis=1)
    # new_df = pd.DataFrame(data, columns=['ID', 'closest_ID'])
    
    return data

In [323]:
flatten_with_key_value = _ddf.map_partitions(mapper, partition_info=None, meta = {'key': 'object', 'flattened_image':'object', 'ID': 'str'})

In [287]:
len(flatten_with_key_value.key)


399969

In [ ]:
similarItems = flatten_with_key_value.reduction(lambda x: x.groupby('key'), aggregate=lambda x: reducer(x))

In [ ]:
g.compute()

In [300]:
# To compute and view the results
result = flatten_with_key_value.compute()
print(result)

          key                flattened_image     ID
0      (0, 1)  [255 255 255 ... 255 255 255]  10000
1      (0, 1)  [255 255 255 ... 255 255 255]  10001
2      (0, 1)  [255 255 255 ... 255 255 255]  10002
3      (0, 1)  [255 255 255 ... 255 255 255]  10003
4      (0, 1)  [255 255 255 ... 255 255 255]  10004
...       ...                            ...    ...
39991  (8, 9)  [255 255 255 ... 255 255 255]   9995
39992  (8, 9)  [255 255 255 ... 255 255 255]   9996
39993  (8, 9)  [255 255 255 ... 255 255 255]   9997
39994  (8, 9)  [255 255 255 ... 255 255 255]   9998
39995  (8, 9)  [255 255 255 ... 255 255 255]   9999

[399969 rows x 3 columns]


In [310]:
g =result.groupby('key')
print(f"Number of reducer: {len(g)}")

Number of reducer: 45


In [314]:
for i in g:
    print(i)

((0, 1),          key                flattened_image     ID
0     (0, 1)  [255 255 255 ... 255 255 255]  10000
1     (0, 1)  [255 255 255 ... 255 255 255]  10001
2     (0, 1)  [255 255 255 ... 255 255 255]  10002
3     (0, 1)  [255 255 255 ... 255 255 255]  10003
4     (0, 1)  [255 255 255 ... 255 255 255]  10004
...      ...                            ...    ...
4439  (0, 1)  [255 255 255 ... 255 255 255]  20145
4440  (0, 1)  [255 255 255 ... 255 255 254]  20146
4441  (0, 1)  [255 255 255 ... 255 255 255]  20147
4442  (0, 1)  [255 255 255 ... 255 255 255]  20148
4443  (0, 1)  [255 255 255 ... 255 255 255]  20149

[8889 rows x 3 columns])
((0, 2),          key                flattened_image     ID
4445  (0, 2)  [255 255 255 ... 255 255 255]  10000
4446  (0, 2)  [255 255 255 ... 255 255 255]  10001
4447  (0, 2)  [255 255 255 ... 255 255 255]  10002
4448  (0, 2)  [255 255 255 ... 255 255 255]  10003
4449  (0, 2)  [255 255 255 ... 255 255 255]  10004
...      ...                          

In [270]:
test_dir = './test'
result.to_csv(f'{test_dir}/key_val.csv', index = False)